In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import os

# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.layer2 = nn.Linear(128, 64)
        self.output_layer = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.dropout(x)
        x = self.relu(self.layer2(x))
        x = self.dropout(x)
        x = self.sigmoid(self.output_layer(x))
        return x

def train_and_evaluate_nn(X_train, y_train, X_test, y_test, description):
    model = NeuralNetwork(X_train.shape[1]).to(device)
    
    # Convert numpy arrays to torch tensors
    X_train_torch = torch.tensor(X_train, dtype=torch.float32)
    y_train_torch = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
    X_test_torch = torch.tensor(X_test, dtype=torch.float32)
    y_test_torch = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
    
    # Create TensorDatasets and DataLoaders with a reduced batch size
    train_dataset = TensorDataset(X_train_torch, y_train_torch)
    test_dataset = TensorDataset(X_test_torch, y_test_torch)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
    
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters())
    
    # Training the model
    for epoch in range(10):  # Loop over the dataset multiple times
        model.train()  # Set model to training mode
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # Validation phase
        model.eval()  # Set model to evaluation mode
        val_running_accuracy = 0.0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                predicted = (outputs > 0.5).float()
                total += labels.size(0)
                val_running_accuracy += (predicted == labels).sum().item()
        
        val_accuracy = val_running_accuracy / total
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}, Val Accuracy: {val_accuracy}')

    
    # Evaluating the model
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = (outputs > 0.5).float()
            predictions.extend(predicted.cpu().numpy())
    
    accuracy = accuracy_score(y_test, np.round(predictions))
    f1 = f1_score(y_test, np.round(predictions))
    
    print(f"Results for {description}:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}\n")

    # Save the model
    model_dir = "Models/Dense_torch"
    description_formatted = description.replace(' ', '_').lower()
    model_filename = os.path.join(model_dir, f"{description_formatted}_model.pth")
    os.makedirs(model_dir, exist_ok=True)
    torch.save(model.state_dict(), model_filename)
    print(f"Model saved as {model_filename}")

Using device: cuda


In [2]:
data_short = np.load('Data/lorfs_100.npz')

X_train_short = data_short['X_train']
X_test_short = data_short['X_test']
y_train_short = data_short['y_train']
y_test_short = data_short['y_test']
X_train_short = X_train_short.reshape(X_train_short.shape[0], -1)
X_test_short = X_test_short.reshape(X_test_short.shape[0], -1)
train_and_evaluate_nn(X_train_short, y_train_short, X_test_short, y_test_short, "Short Dataset")



Epoch 1, Loss: 0.6484804896223032, Val Accuracy: 0.6196188272544602
Epoch 2, Loss: 0.6368818433299422, Val Accuracy: 0.6200298362053218
Epoch 3, Loss: 0.6178977894251685, Val Accuracy: 0.6154174024234306
Epoch 4, Loss: 0.585647831652056, Val Accuracy: 0.6073190038360835
Epoch 5, Loss: 0.5486821695660468, Val Accuracy: 0.6019606649211472
Epoch 6, Loss: 0.5163876424524676, Val Accuracy: 0.5958564208731657
Epoch 7, Loss: 0.48878594304124817, Val Accuracy: 0.595369299153626
Epoch 8, Loss: 0.46629880507849375, Val Accuracy: 0.5892802776593802
Epoch 9, Loss: 0.44613459543938877, Val Accuracy: 0.5838458259757657
Epoch 10, Loss: 0.4286850905485357, Val Accuracy: 0.5838153808682944
Results for Short Dataset:
Accuracy: 0.5838
F1 Score: 0.5495

Model saved as Models/Dense_torch/short_dataset_model.pth


In [3]:
del X_test_short, X_train_short, y_test_short, y_train_short
data_medium = np.load('Data/lorfs_400.npz')

X_train_medium = data_medium['X_train']
X_test_medium = data_medium['X_test']
y_train_medium = data_medium['y_train']
y_test_medium = data_medium['y_test']
X_train_medium = X_train_medium.reshape(X_train_medium.shape[0], -1)
X_test_medium = X_test_medium.reshape(X_test_medium.shape[0], -1)

train_and_evaluate_nn(X_train_medium, y_train_medium, X_test_medium, y_test_medium, "Medium Dataset")


Epoch 1, Loss: 0.38905812932030404, Val Accuracy: 0.8550797604536542
Epoch 2, Loss: 0.3492870307932459, Val Accuracy: 0.852204674333563
Epoch 3, Loss: 0.31598286747650506, Val Accuracy: 0.8500847951666755
Epoch 4, Loss: 0.28976995017053964, Val Accuracy: 0.8477926758174784
Epoch 5, Loss: 0.26690315281223737, Val Accuracy: 0.8449440881869733
Epoch 6, Loss: 0.24355192692713748, Val Accuracy: 0.8406910806084054
Epoch 7, Loss: 0.2232335796077755, Val Accuracy: 0.8366765594361122
Epoch 8, Loss: 0.19992165709718626, Val Accuracy: 0.8314696062324447
Epoch 9, Loss: 0.18245076159911083, Val Accuracy: 0.8206847209709046
Epoch 10, Loss: 0.1649314014763324, Val Accuracy: 0.8233875669086862
Results for Medium Dataset:
Accuracy: 0.8234
F1 Score: 0.8158

Model saved as Models/Dense_torch/medium_dataset_model.pth


In [4]:
del X_test_medium, X_train_medium, y_test_medium, y_train_medium
data_long = np.load('Data/lorfs_1000.npz')

X_train_long = data_long['X_train']
X_test_long = data_long['X_test']
y_train_long = data_long['y_train']
y_test_long = data_long['y_test']
X_train_long = X_train_long.reshape(X_train_long.shape[0], -1)
X_test_long = X_test_long.reshape(X_test_long.shape[0], -1)

train_and_evaluate_nn(X_train_long, y_train_long, X_test_long, y_test_long, "Long Dataset")

Epoch 1, Loss: 0.32344923041830326, Val Accuracy: 0.9001316722374152
Epoch 2, Loss: 0.17476654077818599, Val Accuracy: 0.8943583510584422
Epoch 3, Loss: 0.06296509631889438, Val Accuracy: 0.8928390560113441
Epoch 4, Loss: 0.03183385565778923, Val Accuracy: 0.8934467740301834
Epoch 5, Loss: 0.022815749568893217, Val Accuracy: 0.8953712144231744
Epoch 6, Loss: 0.018237029123544187, Val Accuracy: 0.8898004659171478
Epoch 7, Loss: 0.01879286980285657, Val Accuracy: 0.8946622100678618
Epoch 8, Loss: 0.014587536595964939, Val Accuracy: 0.8930416286842905
Epoch 9, Loss: 0.01783791331252476, Val Accuracy: 0.8970930821432189
Epoch 10, Loss: 0.014525227295797735, Val Accuracy: 0.8951686417502279
Results for Long Dataset:
Accuracy: 0.8952
F1 Score: 0.8990

Model saved as Models/Dense_torch/long_dataset_model.pth
